In [22]:
from _datetime import datetime, timedelta
from gc import set_debug
import pandas as pd
import soupsieve
from Demos.win32cred_demo import attrs
from attr.validators import min_len
from bs4 import BeautifulSoup
import requests
import xlrd as xlrd

In [23]:
url = "https://www.mse.mk/mk/stats/symbolhistory/ADIN"
response = requests.get(url)

In [24]:
response.text

'\r\n\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1">\r\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n    <meta name="description" content="Macedonian Stock Exchange" />\r\n    <meta name="keywords" content="mse, trade, stock exchange, stock, market, macedonia, exchange, share, listed, companies, data, transfer, market, securities, акции, котирани, друштва, berza, borza, берза, македонска, тргување, тргува" />\r\n    <meta name="author" content="Macedonian Stock Exchange" />\r\n    <meta name="copyright" content="Macedonian Stock Exchange" />\r\n    <meta property="og:title" content="Историски податоци" />\r\n    <meta property="og:url" content="/mk/stats/symbolhistory/ADIN" />\r\n    <meta property="og:image" content="https://www.mse.mk/Content/images/mse-logo.png" />\r\n\r\n    \r\n    <li

In [25]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <meta content="Macedonian Stock Exchange" name="description">
    <meta content="mse, trade, stock exchange, stock, market, macedonia, exchange, share, listed, companies, data, transfer, market, securities, акции, котирани, друштва, berza, borza, берза, македонска, тргување, тргува" name="keywords">
     <meta content="Macedonian Stock Exchange" name="author"/>
     <meta content="Macedonian Stock Exchange" name="copyright"/>
     <meta content="Историски податоци" property="og:title"/>
     <meta content="/mk/stats/symbolhistory/ADIN" property="og:url"/>
     <meta content="https://www.mse.mk/Content/images/mse-logo.png" property="og:image"/>
     <link href="data:image/x-icon;base64,AAABAAEAICAAAAEAIACoEAAAFgAAACgAAAAgAAAAQAAAAAE

In [26]:
options = soup.select(".panel .form-control option")
options

[<option selected="selected" value="ADIN">ADIN</option>,
 <option value="ALK">ALK</option>,
 <option value="ALKB">ALKB</option>,
 <option value="AMEH">AMEH</option>,
 <option value="APTK">APTK</option>,
 <option value="ATPP">ATPP</option>,
 <option value="AUMK">AUMK</option>,
 <option value="BANA">BANA</option>,
 <option value="BGOR">BGOR</option>,
 <option value="BIKF">BIKF</option>,
 <option value="BIM">BIM</option>,
 <option value="BLTU">BLTU</option>,
 <option value="CBNG">CBNG</option>,
 <option value="CDHV">CDHV</option>,
 <option value="CEVI">CEVI</option>,
 <option value="CKB">CKB</option>,
 <option value="CKBKO">CKBKO</option>,
 <option value="DEBA">DEBA</option>,
 <option value="DIMI">DIMI</option>,
 <option value="EDST">EDST</option>,
 <option value="ELMA">ELMA</option>,
 <option value="ELNC">ELNC</option>,
 <option value="ENER">ENER</option>,
 <option value="ENSA">ENSA</option>,
 <option value="EUHA">EUHA</option>,
 <option value="EUMK">EUMK</option>,
 <option value="EVRO">

In [27]:
test = soup.select(".panel-body .thead-default th")
headers = []
for i in range(0, len(test)):
    option = test[i]
    option.text.strip()
    headers.append(' '.join(option.text.split()))
    
headers

['Датум',
 'Цена на последна трансакција',
 'Мак.',
 'Мин.',
 'Просечна цена',
 '%пром.',
 'Количина',
 'Промет во БЕСТ во денари',
 'Вкупен промет во денари']

In [53]:
def extraction_of_data(soup):
    extracted_data = []
    data = soup.select("tbody tr")
    for i in range(0, len(data)):
        options = data[i].select("td")
        date = options[0]
        trade_price = options[1]
        max = options[2]
        min = options[3]
        avg = options[4]
        chg = options[5]
        vol = options[6]
        turnover = options[7]
        total = options[8]
    
        dict = {
            "ИМЕ" : valid_code,
            "ДАТУМ" : date.text,
            "T Price" : trade_price.text,
            "МАКС" :  max.text,
            "МИН" : min.text,
            "Пресек" : avg.text,
            "CHG" : chg.text,
            "VOL" : vol.text,
            "Turnover" : turnover.text,
            "TOTAL" : total.text
            
        }
        extracted_data.append(dict)
        
    return extracted_data
    # dataframe = pd.DataFrame(extracted_data)
    # print(dataframe)
    # dataframe.to_csv("dataframe.csv", mode='a')

In [58]:
def iterating_pages(valid_code):
    matrix = []
    for i in range(0, 11):
        today = datetime.today()
        year = today.year
        start_year = year - i
        end_year = year - i
        url = "https://www.mse.mk/mk/stats/symbolhistory/" + valid_code
        response = requests.post(url, data={"FromDate" : "01.01." + str(start_year), "ToDate" : "31.12." + str(end_year)})
        soup = BeautifulSoup(response.text, 'html.parser')
        matrix.extend(extraction_of_data(soup))
        
    dataframe = pd.DataFrame(matrix)
    dataframe.to_csv("dataframe_" + valid_code + ".csv")

In [59]:
for option in options:
    valid_code = (option["value"].strip())
    if valid_code and not any(char.isdigit() for char in valid_code):
        iterating_pages(valid_code)